# Cleaning the data

## 1) Joining
At the moment, I have two different tables with the data, depending on their source, and I need to join them to have all the information in one dataframe

In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
pd.set_option("max_columns", 100)

In [2]:
steam_app_data = pd.read_csv('data_backup/steam_app_data.csv') #Backup of the data created while the final version downloads
steam_app_data

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 819, 'final': 8...","[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 118421},NaN,"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 4479},NaN,"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 3122},NaN,"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum':

In [3]:
steamspy_data = pd.read_csv('data_backup/steamspy_data.csv') #Backup of the data created while the final version downloads
steamspy_data

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,192940,4936,0,"10,000,000 .. 20,000,000",11368,157,193,48,999,999,0,"English, French, German, Italian, Spanish - Sp...",Action,17108,"{'Action': 5379, 'FPS': 4801, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,5409,895,0,"5,000,000 .. 10,000,000",217,58,14,58,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,114,"{'Action': 745, 'FPS': 306, 'Multiplayer': 257..."
2,30,Day of Defeat,Valve,Valve,NaN,5000,555,0,"5,000,000 .. 10,000,000",469,0,24,0,499,499,0,"English, French, German, Italian, Spanish - Spain",Action,157,"{'FPS': 786, 'World War II': 247, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1852,412,0,"5,000,000 .. 10,000,000",256,0,13,0,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,5,"{'Action': 629, 'FPS': 139, 'Classic': 107, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,13243,661,0,"5,000,000 .. 10,000,000",370,0,213,0,499,499,0,"English, French, German, Korean",Action,157,"{'FPS': 880, 'Action': 321, 'Classic': 250, 'S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1551360,Forza Horizon 5,Playground Games,Xbox Game Studios,NaN,53313,8359,0,"1,000,000 .. 2,000,000",2806,355,1786,394,5999,5999,0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Racing, Simulation, Sports",16779,"{'Racing': 471, 'Open World': 364, 'Driving': ..."
996,1593500,God of War,Santa Monica Studio,PlayStation PC LLC,NaN,34394,1053,0,"1,000,000 .. 2,000,000",1188,620,1092,524,4999,4999,0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, RPG",23121,"{'Action': 347, 'Adventure': 323, 'Story Rich'..."
997,1627140,Sabre Team,Krisalis Software,Ziggurat,NaN,0,1,0,"100,000 .. 200,000",0,0,0,0,699,699,0,"English, French, Italian, German, Spanish - Spain","RPG, Strategy",0,"{'Turn-Based Tactics': 195, 'Military': 188, '..."
998,1794680,Vampire Survivors,poncle,poncle,NaN,29794,262,0,"2,000,000 .. 5,000,000",583,487,610,479,299,299,0,English,"Action, Casual, Indie, RPG, Early Access",60974,"{'Action Roguelike': 131, 'Pixel Graphics': 11..."


The way both APIs are structured make some of the columns to be repeated between the tables, but other than that, the should be able to merge without issue.

In [4]:
data = pd.concat([steam_app_data, steamspy_data], axis=1)
data

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 819, 'final': 8...","[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 118421},NaN,"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",10,Counter-Strike,Valve,Valve,NaN,192940,4936,0,"10,000,000 .. 20,000,000",11368,157,193,48,999,999,0,"English, French, German, Italian, Spanish - Sp...",Action,17108,"{'Action': 5379, 'FPS': 4801, 'Multiplayer': 3..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 4479},NaN,"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",20,Team Fortress Classic,Valve,Valve,NaN,5409,895,0,"5,000,000 .. 10,000,000",217,58,14,58,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,114,"{'Action': 745, 'FPS': 306, 'Multiplayer': 257..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 

## 2) Removing columns
Besides finding the duplicate columns, there are other columns (like the descriptions) that offer no value to me, so I will clean them from the table to make it lighter.

First I will look at the columns with descriptions, images and the like:

In [5]:
cols_to_drop = ['detailed_description', 'about_the_game', 'short_description', 'header_image', 'website', 'pc_requirements',
               'mac_requirements', 'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
               'packages', 'package_groups', 'screenshots', 'movies', 'achievements', 'support_info', 'background', 'reviews']

In [6]:
data = data.drop(cols_to_drop, 1)

C:\Users\carlo\AppData\Local\Temp/ipykernel_28408/2397828251.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(cols_to_drop, 1)


In [7]:
data.head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,fullgame,supported_languages,developers,publishers,demos,price_overview,platforms,metacritic,categories,genres,recommendations,release_date,content_descriptors,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,game,Counter-Strike,10,0.0,False,NaN,NaN,NaN,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 819, 'final': 8...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 118421},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'ids': [2, 5], 'notes': 'Includes intense vio...",10,Counter-Strike,Valve,Valve,NaN,192940,4936,0,"10,000,000 .. 20,000,000",11368,157,193,48,999,999,0,"English, French, German, Italian, Spanish - Sp...",Action,17108,"{'Action': 5379, 'FPS': 4801, 'Multiplayer': 3..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,NaN,"English, French, German, Italian, Spanish - Sp...",['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 399, 'final': 3...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 4479},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'ids': [2, 5], 'notes': 'Includes intense vio...",20,Team Fortress Classic,Valve,Valve,NaN,5409,895,0,"5,000,000 .. 10,000,000",217,58,14,58,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,114,"{'Action': 745, 'FPS': 306, 'Multiplayer': 257..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,NaN,"English, French, German, Italian, Spanish - Spain",['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 399, 'final': 3...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 3122},"{'coming_soon': False, 'date': '1 May, 2003'}","{'ids': [], 'notes': None}",30,Day of Defeat,Valve,Valve,NaN,5000,555,0,"5,000,000 .. 10,000,000",469,0,24,0,499,499,0,"English, French, German, Italian, Spanish - Spain",Action,157,"{'FPS': 786, 'World War II': 247, 'Multiplayer..."
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,NaN,"English, French, German, Italian, Spanish - Sp...",['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 399, 'final': 3...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 1496},"{'coming_soon': False, 'date': '1 Jun, 2001'}","{'ids': [], 'notes': None}",40,Deathmatch Classic,Valve,Valve,NaN,1852,412,0,"5,000,000 .. 10,000,000",256,0,13,0,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,5,"{'Action': 629, 'FPS': 139, 'Classic': 107, 'M..."
4,game,Half-Life: Opposing Force,50,0.0,False,NaN,NaN,NaN,"English, French, German, Korean",['Gearbox Software'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 399, 'final': 3...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",{'total': 11308},"{'coming_soon': False, 'date': '1 Nov, 1999'}","{'ids': [], 'notes': None}",50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,13243,661,0,"5,000,000 .. 10,000,000",370,0,213,0,499,499,0,"English, French, German, Korean",Action,157,"{'FPS': 880, 'Action': 321, 'Classic': 250, 'S..."


Now I will check the duplicates:

In [8]:
dupes_to_drop = ['name', 'steam_appid', 'supported_languages','developer', 'publisher', 'genre']
data = data.drop(dupes_to_drop, 1)

C:\Users\carlo\AppData\Local\Temp/ipykernel_28408/1596603079.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(dupes_to_drop, 1)


As there are two columns with the name `name`, both are dropped. As they work as identifiers, their use was limited anyway.

In [9]:
data.shape

(1000, 33)

## 3) Analyzing rows
I want to check the existence of duplicates:

In [10]:
data[data.duplicated()]

,type,required_age,is_free,controller_support,dlc,fullgame,developers,publishers,demos,price_overview,platforms,metacritic,categories,genres,recommendations,release_date,content_descriptors,appid,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,ccu,tags


With the sample code, there are no duplicates.

## 4) Analyzing columns
At first glance, I can already see there are various fields with NaN values, and other fields with lists or dictionaries that should be transformed into individual columns.

### NaNs


In [11]:
data.isna().sum()

type                      2
required_age              2
is_free                   2
controller_support      629
dlc                     462
fullgame               1000
developers                3
publishers                2
demos                   894
price_overview          289
platforms                 2
metacritic              415
categories                4
genres                    5
recommendations         104
release_date              2
content_descriptors       2
appid                     0
score_rank             1000
positive                  0
negative                  0
userscore                 0
owners                    0
average_forever           0
average_2weeks            0
median_forever            0
median_2weeks             0
price                     0
initialprice              0
discount                  0
languages                 0
ccu                       0
tags                      0
dtype: int64

I see that (in the sample version of the dataset) the amount of NaN values in the columns `fullgame` and `score_rank` are too high to keep them.

In [12]:
data = data.drop(['fullgame', 'score_rank'], axis=1)

The column `type` works as a flag to check if the row has actual data or is an empty row: if it is NaN, the rest of the row will also be empty besides the appid.

In [13]:
data[data['type'].isna()]

,type,required_age,is_free,controller_support,dlc,developers,publishers,demos,price_overview,platforms,metacritic,categories,genres,recommendations,release_date,content_descriptors,appid,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,ccu,tags
133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32400,2110,258,0,"500,000 .. 1,000,000",94,0,23,0,599,599,0,"English, French, German, Italian, Spanish - Spain",20,"{'FPS': 145, 'Action': 133, 'Classic': 117, 'S..."
328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,231060,1783,1236,0,"1,000,000 .. 2,000,000",169,0,98,0,0,0,0,"English, Portuguese - Brazil, Traditional Chinese",15,"{'Free to Play': 241, 'RPG': 157, 'MMORPG': 12..."


In [14]:
data.dropna(subset = ["type"], inplace=True)

Some of the columns with NaNs can have those values changed into Unknown, but those columns may have the above mentioned lists and dictionaries.  
I will transform those values as stated.

In [15]:
data['developers'] = data['developers'].fillna('Unknown')
data['publishers'] = data['publishers'].fillna('Unknown')
data['categories'] = data['categories'].fillna('Unknown')
data['content_descriptors'] = data['content_descriptors'].fillna('Unknown')


Further fillings require studying the values on those columns:

In [16]:
data['required_age'].value_counts(dropna=False)

0.0     818
18.0    109
16.0     38
17.0     20
12.0      7
15.0      6
Name: required_age, dtype: int64

These NaNs can be changed into 0

In [17]:
data['required_age'] = data['required_age'].fillna(0)

In [18]:
data['is_free'].value_counts(dropna=False)

False    782
True     216
Name: is_free, dtype: int64

I can change the NaNs into false, as it is the most common value this column will have, but I can also change it into 0 and 1 already.

In [19]:
data['is_free'] = data['is_free'].fillna(0)
data['is_free'] = np.where(data['is_free'] == True, 1, 0)

In [20]:
data['is_free'].value_counts(dropna=False)

0    782
1    216
Name: is_free, dtype: int64

In [21]:
data['controller_support'].value_counts(dropna=False)

NaN     627
full    371
Name: controller_support, dtype: int64

This column may pressent issues when using the whole set, but the idea is to make it boolean (0 or 1).

In [22]:
data['controller_support'] = data['controller_support'].fillna(0)
data['controller_support'] = np.where(data['controller_support'] == 'full', 1, 0)

In [23]:
data['controller_support'].value_counts(dropna=False)

0    627
1    371
Name: controller_support, dtype: int64

In [24]:
data['dlc'].value_counts(dropna=False)

NaN                                                                                 460
[42719, 42722, 42718, 42716]                                                          2
[865670]                                                                              1
[514800]                                                                              1
[741350, 1071200, 1804460]                                                            1
                                                                                   ... 
[256320, 256321]                                                                      1
[323410]                                                                              1
[554000, 314680, 366080, 467300, 584380, 584381, 584382, 584383, 698890, 379400]      1
[386860, 413160, 982110]                                                              1
[1881490, 1858303, 1858305, 1867630, 1858300, 1858301]                                1
Name: dlc, Length: 538, dtype: i

This column represents whether the game has any extra downloadable content, and links it to its appid. I just need to know if there is, so I will change it to boolean as well.

In [25]:
data['dlc'] = data['dlc'].fillna(0)
data['dlc'] = np.where(data['dlc'] == 0, 0, 1)

In [26]:
data['dlc'].value_counts(dropna=False)

1    538
0    460
Name: dlc, dtype: int64

In [27]:
data['demos'].value_counts(dropna=False)

NaN                                                                  892
[{'appid': 47760, 'description': ''}]                                  2
[{'appid': 737770, 'description': ''}]                                 1
[{'appid': 1210810, 'description': ''}]                                1
[{'appid': 543080, 'description': 'Demo'}]                             1
                                                                    ... 
[{'appid': 204980, 'description': ''}]                                 1
[{'appid': 48710, 'description': ''}]                                  1
[{'appid': 48010, 'description': ''}]                                  1
[{'appid': 248140, 'description': ''}]                                 1
[{'appid': 1504980, 'description': "It Takes Two Friend's Pass"}]      1
Name: demos, Length: 106, dtype: int64

As above, for this column I only want to know if there is a demo associated to the game, not the ID of said demo, so I will do the same process.

In [28]:
data['demos'] = data['demos'].fillna(0)
data['demos'] = np.where(data['demos'] == 0, 0, 1)

In [29]:
data['demos'].value_counts(dropna=False)

0    892
1    106
Name: demos, dtype: int64

In [30]:
data['price_overview'].value_counts(dropna=False)

NaN                                                                                                                                      287
{'currency': 'EUR', 'initial': 1999, 'final': 1999, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '19,99€'}         108
{'currency': 'EUR', 'initial': 2999, 'final': 2999, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '29,99€'}          58
{'currency': 'EUR', 'initial': 999, 'final': 999, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '9,99€'}             55
{'currency': 'EUR', 'initial': 1499, 'final': 1499, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '14,99€'}          54
                                                                                                                                        ... 
{'currency': 'EUR', 'initial': 2500, 'final': 2500, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '25,--€'}           1
{'currency': 

This is too complex to handle at the moment, but I may need it later.

In [31]:
data['platforms'].value_counts(dropna=False)

{'windows': True, 'mac': False, 'linux': False}    606
{'windows': True, 'mac': True, 'linux': True}      253
{'windows': True, 'mac': True, 'linux': False}     118
{'windows': True, 'mac': False, 'linux': True}      21
Name: platforms, dtype: int64

I will assign the most common entry (available on windows) to the NaN values, and I will translate them into columns later.


In [32]:
data['platforms'] = data['platforms'].fillna("{'windows': True, 'mac': False, 'linux': False}")

In [33]:
data['platforms'].value_counts(dropna=False)

{'windows': True, 'mac': False, 'linux': False}    606
{'windows': True, 'mac': True, 'linux': True}      253
{'windows': True, 'mac': True, 'linux': False}     118
{'windows': True, 'mac': False, 'linux': True}      21
Name: platforms, dtype: int64

In [34]:
data['metacritic'].value_counts(dropna=False)

NaN                                                                                                             413
{'score': 88, 'url': 'https://www.metacritic.com/game/pc/fear?ftag=MCD-06-10aaa1f'}                               3
{'score': 94, 'url': 'https://www.metacritic.com/game/pc/mass-effect-2?ftag=MCD-06-10aaa1f'}                      2
{'score': 94, 'url': 'https://www.metacritic.com/game/pc/the-elder-scrolls-iv-oblivion?ftag=MCD-06-10aaa1f'}      2
{'score': 91, 'url': 'https://www.metacritic.com/game/pc/dragon-age-origins?ftag=MCD-06-10aaa1f'}                 2
                                                                                                               ... 
{'score': 90, 'url': 'https://www.metacritic.com/game/pc/total-war-shogun-2?ftag=MCD-06-10aaa1f'}                 1
{'score': 88, 'url': 'https://www.metacritic.com/game/pc/torchlight-ii?ftag=MCD-06-10aaa1f'}                      1
{'score': 89, 'url': 'https://www.metacritic.com/game/pc/xcom-enemy-unkn

I want to keep as much information from this column as possible, as it is one of the most important sources of information for this dataset. To keep the rows with NaN values, I will assign a score of 0 and hope it doesn't affect the results.

In [52]:
metacritic_nan = "{'score': 0, 'url': ''}"
data['metacritic'] = data['metacritic'].fillna(metacritic_nan)

In [35]:
data['genres'].value_counts(dropna=False)

[{'id': '1', 'description': 'Action'}]                                                                                                                                                                                                                                                       148
[{'id': '1', 'description': 'Action'}, {'id': '25', 'description': 'Adventure'}]                                                                                                                                                                                                              83
[{'id': '2', 'description': 'Strategy'}]                                                                                                                                                                                                                                                      42
[{'id': '1', 'description': 'Action'}, {'id': '3', 'description': 'RPG'}]                                                            

I want to keep as much information from this column as possible, even though it is the most likely to be dropped due to complexity. For that, I will create a fallback dictionary with an Unknown genre and assign it to the NaN values. Later, that Unknown will also be dropped, when the processing of this column finishes.

In [50]:
unknown_genre = "[{'id': '0', 'description': 'Unknown'}]"
data['genres'] = data['genres'].fillna(unknown_genre)

In [36]:
data['recommendations'].value_counts(dropna=False)

NaN                  102
{'total': 139}         4
{'total': 7511}        3
{'total': 206}         3
{'total': 321}         3
                    ... 
{'total': 6120}        1
{'total': 7870}        1
{'total': 394511}      1
{'total': 2494}        1
{'total': 24139}       1
Name: recommendations, Length: 875, dtype: int64

I will need this information, so I will assign {'total': 0} as value. (I don't trust this result yet)

In [37]:
data['recommendations'] = data['recommendations'].fillna("{'total': 0}")

In [38]:
data['recommendations'].value_counts(dropna=False)

{'total': 0}         102
{'total': 139}         4
{'total': 7511}        3
{'total': 206}         3
{'total': 321}         3
                    ... 
{'total': 6120}        1
{'total': 7870}        1
{'total': 394511}      1
{'total': 2494}        1
{'total': 24139}       1
Name: recommendations, Length: 875, dtype: int64

In [39]:
data['release_date'].value_counts(dropna=False)

{'coming_soon': False, 'date': ''}                13
{'coming_soon': False, 'date': '18 Jun, 2020'}     4
{'coming_soon': False, 'date': '11 Jun, 2020'}     4
{'coming_soon': False, 'date': '22 Aug, 2013'}     4
{'coming_soon': False, 'date': '27 Oct, 2016'}     4
                                                  ..
{'coming_soon': False, 'date': 'Aug 13, 2013'}     1
{'coming_soon': False, 'date': '11 Mar, 2016'}     1
{'coming_soon': False, 'date': '2 Jul, 2014'}      1
{'coming_soon': False, 'date': '20 May, 2014'}     1
{'coming_soon': False, 'date': '20 Jan, 2022'}     1
Name: release_date, Length: 829, dtype: int64

In [53]:
data.isna().sum()

type                     0
required_age             0
is_free                  0
controller_support       0
dlc                      0
developers               0
publishers               0
demos                    0
price_overview         287
platforms                0
metacritic               0
categories               0
genres                   0
recommendations          0
release_date             0
content_descriptors      0
appid                    0
positive                 0
negative                 0
userscore                0
owners                   0
average_forever          0
average_2weeks           0
median_forever           0
median_2weeks            0
price                    0
initialprice             0
discount                 0
languages                0
ccu                      0
tags                     0
dtype: int64

In [41]:
test_data = data.copy()

In [42]:
type(data['release_date'][0])

str

In [43]:
import ast

res = ast.literal_eval(data['release_date'][0])
res

{'coming_soon': False, 'date': '1 Nov, 2000'}

In [44]:
type(res)

dict

In [45]:
df = test_data['release_date'].map(ast.literal_eval)

In [46]:
df

0       {'coming_soon': False, 'date': '1 Nov, 2000'}
1       {'coming_soon': False, 'date': '1 Apr, 1999'}
2       {'coming_soon': False, 'date': '1 May, 2003'}
3       {'coming_soon': False, 'date': '1 Jun, 2001'}
4       {'coming_soon': False, 'date': '1 Nov, 1999'}
                            ...                      
995     {'coming_soon': False, 'date': '8 Nov, 2021'}
996    {'coming_soon': False, 'date': '14 Jan, 2022'}
997     {'coming_soon': False, 'date': '4 Jun, 2021'}
998    {'coming_soon': False, 'date': '17 Dec, 2021'}
999    {'coming_soon': False, 'date': '20 Jan, 2022'}
Name: release_date, Length: 998, dtype: object

In [47]:
type(test_data['release_date'][0])

str

In [48]:
pd.DataFrame.from_dict(test_data['release_date'], orient='tight')

ValueError: only recognize index or columns for orient

In [ ]:
df.apply(pd.Series)